In [2]:
import numpy as np
import pandas as pd
import mygene
import omnipath as op
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import mellon as ml
from pyensembl import EnsemblRelease
import random
import netrd
import networkx as nx

c:\Users\melis\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import gtex expression data

gtex_link = 'raw_data/GTEx_tissue_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'
exp = pd.read_csv(gtex_link, sep='\t', index_col='Description', skiprows=2)

exp_cns = exp.loc[:, ['Brain - Amygdala',
       'Brain - Anterior cingulate cortex (BA24)',
       'Brain - Caudate (basal ganglia)', 'Brain - Cerebellar Hemisphere',
       'Brain - Cerebellum', 'Brain - Cortex', 'Brain - Frontal Cortex (BA9)',
       'Brain - Hippocampus', 'Brain - Hypothalamus',
       'Brain - Nucleus accumbens (basal ganglia)',
       'Brain - Putamen (basal ganglia)', 'Brain - Spinal cord (cervical c-1)',
       'Brain - Substantia nigra']]
exp_cns = exp_cns.loc[(exp_cns > 0).any(axis=1)]

# import omnipath db
db = op.interactions.import_intercell_network(transmitter_params = {"categories":"ligand"}, receiver_params = {"categories": "receptor"})
db = db[np.logical_not(db['genesymbol_intercell_source'].str.startswith('HLA'))]
db = db[np.logical_not(db['genesymbol_intercell_target'].str.startswith('HLA'))]
db = db[~db['genesymbol_intercell_target'].astype(str).str.startswith('COMPLEX')]
db = db[~db['genesymbol_intercell_source'].astype(str).str.startswith('COMPLEX')]

# specify the release number
ensembl = EnsemblRelease(109)
genes = ensembl.genes()

mg = mygene.MyGeneInfo()
ensembl_gtex = list(np.unique(pd.DataFrame(list(exp_cns.index.str.split('.')))[0]))

# create a DataFrame of gene information
ensembl_ann = pd.DataFrame({
    'gene_id': [gene.gene_id for gene in genes],
    'gene_name': [gene.gene_name for gene in genes],
    'chromosome': [gene.contig for gene in genes],
    'type': [gene.biotype for gene in genes]
})
gtex_annotated = ensembl_ann[ensembl_ann['gene_name'].isin(ensembl_gtex)] # take list of genes expressed in brain
protein = gtex_annotated[gtex_annotated['type']=='protein_coding']        # take protein-expressing ones
protein = protein.rename(columns={'gene_name':'gene'}).set_index('gene', drop=True)


In [4]:
all_source, pctsrc, all_targets, pcttgt, summary_universe_LR = ml.lr.find_hits(protein,db)  # of these proteins, which LRs are expressed?
universe_LRs = list(pd.concat([all_source['gene'],all_targets['gene']]).unique()) # define universe of LRs

In [5]:
# 
reference = pd.read_csv('processed_data/hot-encoded-diseases_0.400000.csv', index_col=0)                     # take actual disease genes
df = reference[reference.index.isin(list(ensembl_ann[ensembl_ann['type']=='protein_coding']['gene_name']))]  # take protein coding ones
df = df.reset_index().rename(columns={'genes':'gene'}).set_index('gene', drop = True)
all_source, pctsrc, all_targets, pcttgt, summary_LR = ml.lr.find_hits(df,db)    
disease_LRs = summary_LR[summary_LR['source+target']!='None']              
reference = reference[reference.index.isin(list(disease_LRs['gene']))]                                        # filter reference to only include LRs
disease_list = list(reference.columns)
#universe_LRs = list(set(universe_LRs) - (set(universe_LRs) & set(disease_list))) # Remove from universe any disease-associated proteins
iter=10
count = 0
all_LR_intersects = pd.DataFrame(np.zeros(shape=(iter, pow(len(disease_list), 2))))
all_LRI_intersects = pd.DataFrame(np.zeros(shape=(iter, pow(len(disease_list), 2))))
all_distances = pd.DataFrame(np.zeros(shape=(iter, pow(len(disease_list), 2))))

for disease1 in disease_list:
    for disease2 in disease_list:
            all_LR_intersects.rename(columns = {count : disease1+'/'+disease2}, inplace = True)
            all_LRI_intersects.rename(columns = {count : disease1+'/'+disease2}, inplace = True)
            all_distances.rename(columns = {count : disease1+'/'+disease2}, inplace = True)
            if (disease2+'/'+disease1) in all_distances.columns:
                all_LR_intersects[disease1+'/'+disease2] = all_LR_intersects[disease2+'/'+disease1]
                all_LRI_intersects[disease1+'/'+disease2] = all_LRI_intersects[disease2+'/'+disease1]
                all_distances[disease1+'/'+disease2] = all_distances[disease2+'/'+disease1]
            else:
                for i in range(0,iter):
                    # Create random list comparable to each disease
                    rand_disease1 = random.sample(list(range(0,len(universe_LRs))), int(reference[disease1].sum()))
                    rand_disease2 = random.sample(list(range(0,len(universe_LRs))), int(reference[disease2].sum()))
                    genes_disease1 = list(pd.DataFrame(universe_LRs).iloc[rand_disease1][0])
                    genes_disease2 = list(pd.DataFrame(universe_LRs).iloc[rand_disease2][0])

                    ### How many overlapping LRs do random disease 1 and random disease 2 share?
                    # Keep only genes with known LR role
                    #genes_disease1 = list(genes_disease1[genes_disease1.isin(list(db['genesymbol_intercell_source'])) | genes_disease1.isin(list(db['genesymbol_intercell_target']))].dropna())
                    #genes_disease2 = list(genes_disease2[genes_disease2.isin(list(db['genesymbol_intercell_source'])) | genes_disease2.isin(list(db['genesymbol_intercell_target']))].dropna())

                    # Check intersect of lists, add total number to all_intersects list
                    LR_intersect = len(list(set(genes_disease1) & set(genes_disease2)))
                    all_LR_intersects.iloc[i, count] = LR_intersect
                    
                    ### How many LRIs do random disease 1 and random disease 2 share?
                    # Get all LRIs where at least one is linked to diseases 
                    LRs_disease1 = db[['genesymbol_intercell_source','genesymbol_intercell_target']][(db['genesymbol_intercell_source'].isin(list(genes_disease1))) | (db['genesymbol_intercell_target'].isin(list(genes_disease1)))]
                    LRs_disease2 = db[['genesymbol_intercell_source','genesymbol_intercell_target']][(db['genesymbol_intercell_source'].isin(list(genes_disease2))) | (db['genesymbol_intercell_target'].isin(list(genes_disease2)))]
                    LRs_disease1 = LRs_disease1.dropna()
                    LRs_disease2 = LRs_disease2.dropna()
                    LRs_disease1_list = list(LRs_disease1['genesymbol_intercell_source']+LRs_disease1['genesymbol_intercell_target'])
                    LRs_disease2_list = list(LRs_disease2['genesymbol_intercell_source']+LRs_disease2['genesymbol_intercell_target'])
                    
                    # Check intersect of lists
                    LRI_intersect = len(list(set(LRs_disease1_list) & set(LRs_disease2_list)))
                    all_LRI_intersects.iloc[i, count] = LRI_intersect

                    ### How close are the random LR networks?
                    # Build an adjacency matrix per disease
                    unique_LRs_disease1 = pd.DataFrame(pd.concat([LRs_disease1['genesymbol_intercell_source'],LRs_disease1['genesymbol_intercell_target']], axis = 0).unique())
                    unique_LRs_disease2 = pd.DataFrame(pd.concat([LRs_disease2['genesymbol_intercell_source'],LRs_disease2['genesymbol_intercell_target']], axis = 0).unique())
                    both_unique = pd.concat([unique_LRs_disease1, unique_LRs_disease2], axis = 0)[0].unique()
                    # For deltacon comparison, the axes of the adjacency matrix must be the same
                    adj_LRs_disease1 = pd.DataFrame(np.zeros(shape=(len(both_unique), len(both_unique))), index = both_unique, columns = both_unique)
                    adj_LRs_disease2 = pd.DataFrame(np.zeros(shape=(len(both_unique), len(both_unique))), index = both_unique, columns = both_unique)
                    for n in range (0, np.shape(LRs_disease1)[0]):
                        adj_LRs_disease1.iloc[adj_LRs_disease1.index.get_loc(LRs_disease1.iloc[n]['genesymbol_intercell_source']), adj_LRs_disease1.columns.get_loc(LRs_disease1.iloc[n]['genesymbol_intercell_target'])] += 1
                    for n in range (0, np.shape(LRs_disease2)[0]):
                        adj_LRs_disease2.iloc[adj_LRs_disease2.index.get_loc(LRs_disease2.iloc[n]['genesymbol_intercell_source']), adj_LRs_disease2.columns.get_loc(LRs_disease2.iloc[n]['genesymbol_intercell_target'])] += 1
                    if (len(LRs_disease1)<1) or (len(LRs_disease2)<1):
                        all_distances.iloc[i, count] = 99
                    else:
                        # Create network and detect communities
                        G1 = nx.from_pandas_adjacency(adj_LRs_disease1)
                        G2 = nx.from_pandas_adjacency(adj_LRs_disease2)
                        # Calculate network distance
                        delta = netrd.distance.DeltaCon()
                        distance = delta.dist(G1, G2)

                        all_distances.iloc[i, count] = distance
            count+=1 

#all_LR_intersects.to_csv('processed_data/02a_bootstrapped_disease_comparisons/bootstrapped_LRs_disease_comparisons.csv')
#all_LRI_intersects.to_csv('processed_data/02a_bootstrapped_disease_comparisons/bootstrapped_LRIs_disease_comparisons.csv')
#all_distances.to_csv('processed_data/02a_bootstrapped_disease_comparisons/bootstrapped_distances_disease_comparisons.csv')


In [6]:
reference = pd.read_csv('processed_data/hot-encoded-diseases_0.400000.csv', index_col=0)
df = reference[reference.index.isin(list(ensembl_ann[ensembl_ann['type']=='protein_coding']['gene_name']))]
df = df.reset_index().rename(columns={'genes':'gene'}).set_index('gene', drop = True)
all_source, pctsrc, all_targets, pcttgt, summary_LR = ml.lr.find_hits(df,db)
reference = reference[reference.index.isin(list(summary_LR['gene']))] # filter reference to only include LRs
disease_list = list(reference.columns)
iter=1
count = 0

iter=1
count = 0
real_LR_intersects = pd.DataFrame(np.zeros(shape=(iter, pow(len(disease_list), 2))))
real_LRI_intersects = pd.DataFrame(np.zeros(shape=(iter, pow(len(disease_list), 2))))
real_distances = pd.DataFrame(np.zeros(shape=(iter, pow(len(disease_list), 2))))

for disease1 in disease_list:
    for disease2 in disease_list:
            real_LR_intersects.rename(columns = {count : disease1+'/'+disease2}, inplace = True)
            real_LRI_intersects.rename(columns = {count : disease1+'/'+disease2}, inplace = True)
            real_distances.rename(columns = {count : disease1+'/'+disease2}, inplace = True)
            for i in range(0,iter):
                # Create random list comparable to each disease
                genes_disease1 = pd.DataFrame(reference[reference[disease1]==1].index)
                genes_disease2 = pd.DataFrame(reference[reference[disease2]==1].index)

                ### How many overlapping LRs do random disease 1 and random disease 2 share?
                # Keep only genes with known LR role
                genes_disease1 = list(genes_disease1[genes_disease1.isin(list(db['genesymbol_intercell_source'])) | genes_disease1.isin(list(db['genesymbol_intercell_target']))].dropna()['genes'])
                genes_disease2 = list(genes_disease2[genes_disease2.isin(list(db['genesymbol_intercell_source'])) | genes_disease2.isin(list(db['genesymbol_intercell_target']))].dropna()['genes'])

                # Check intersect of lists, add total number to all_intersects list
                LR_intersect = len(list(set(genes_disease1) & set(genes_disease2)))
                real_LR_intersects.iloc[i, count] = LR_intersect
                
                ### How many LRIs do random disease 1 and random disease 2 share?
                # Get all LRIs where at least one is linked to diseases 
                LRs_disease1 = db[['genesymbol_intercell_source','genesymbol_intercell_target']][(db['genesymbol_intercell_source'].isin(list(genes_disease1))) | (db['genesymbol_intercell_target'].isin(list(genes_disease1)))]
                LRs_disease2 = db[['genesymbol_intercell_source','genesymbol_intercell_target']][(db['genesymbol_intercell_source'].isin(list(genes_disease2))) | (db['genesymbol_intercell_target'].isin(list(genes_disease2)))]
                LRs_disease1 = LRs_disease1.dropna()
                LRs_disease2 = LRs_disease2.dropna()
                LRs_disease1_list = list(LRs_disease1['genesymbol_intercell_source']+LRs_disease1['genesymbol_intercell_target'])
                LRs_disease2_list = list(LRs_disease2['genesymbol_intercell_source']+LRs_disease2['genesymbol_intercell_target'])
                
                # Check intersect of lists
                LRI_intersect = len(list(set(LRs_disease1_list) & set(LRs_disease2_list)))
                real_LRI_intersects.iloc[i, count] = LRI_intersect

                ### How close are the random LR networks?
                # Build an adjacency matrix per disease
                unique_LRs_disease1 = pd.DataFrame(pd.concat([LRs_disease1['genesymbol_intercell_source'],LRs_disease1['genesymbol_intercell_target']], axis = 0).unique())
                unique_LRs_disease2 = pd.DataFrame(pd.concat([LRs_disease2['genesymbol_intercell_source'],LRs_disease2['genesymbol_intercell_target']], axis = 0).unique())
                both_unique = pd.concat([unique_LRs_disease1, unique_LRs_disease2], axis = 0)[0].unique()
                # For deltacon comparison, the axes of the adjacency matrix must be the same
                adj_LRs_disease1 = pd.DataFrame(np.zeros(shape=(len(both_unique), len(both_unique))), index = both_unique, columns = both_unique)
                adj_LRs_disease2 = pd.DataFrame(np.zeros(shape=(len(both_unique), len(both_unique))), index = both_unique, columns = both_unique)
                for n in range (0, np.shape(LRs_disease1)[0]):
                    adj_LRs_disease1.iloc[adj_LRs_disease1.index.get_loc(LRs_disease1.iloc[n]['genesymbol_intercell_source']), adj_LRs_disease1.columns.get_loc(LRs_disease1.iloc[n]['genesymbol_intercell_target'])] += 1
                for n in range (0, np.shape(LRs_disease2)[0]):
                    adj_LRs_disease2.iloc[adj_LRs_disease2.index.get_loc(LRs_disease2.iloc[n]['genesymbol_intercell_source']), adj_LRs_disease2.columns.get_loc(LRs_disease2.iloc[n]['genesymbol_intercell_target'])] += 1
                if (len(LRs_disease1)<1) or (len(LRs_disease2)<1):
                    real_distances.iloc[i, count] = 99
                else:
                    # Create network and detect communities
                    G1 = nx.from_pandas_adjacency(adj_LRs_disease1)
                    G2 = nx.from_pandas_adjacency(adj_LRs_disease2)
                    # Calculate network distance
                    delta = netrd.distance.DeltaCon()
                    distance = delta.dist(G1, G2)

                    real_distances.iloc[i, count] = distance
            count+=1 

In [21]:
bla = all_distances.columns[80]

In [25]:
(all_LR_intersects[all_distances.columns[80]]>real_LR_intersects[all_distances.columns[80]][0]).sum()

0

In [31]:
real_LR_intersects[all_distances.columns[80]][0]==0

if (all_LR_intersects[all_distances.columns[80]]==0) and (real_LR_intersects[all_distances.columns[80]][0]==0):
    print('ZERO')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [38]:
(all_LR_intersects[all_distances.columns[80]]>real_LR_intersects[all_distances.columns[80]][0]).sum()/10

0.1

In [42]:
all_LR_intersects[all_distances.columns[80]].sum

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: BrainAneurysm/MigraineDisorder, dtype: float64

In [47]:
pvals_LR = []
pvals_LRI = []
pvals_dist = []

for pair in list(all_distances.columns):
    if all_LR_intersects[pair].sum()==0:
        pvals_LR.append(1)
    else:
        pvals_LR.append((all_LR_intersects[pair]>real_LR_intersects[pair][0]).sum()/10)
    if all_LRI_intersects[pair].sum()==0:
        pvals_LRI.append(1)
    else:
        pvals_LRI.append((all_LRI_intersects[pair]>real_LRI_intersects[pair][0]).sum()/10)
    if all_distances[pair].sum()==0:
        pvals_dist.append(1)
    else:
        pvals_dist.append((all_distances[pair]>real_distances[pair][0]).sum()/10)

In [ ]:
pvals_LR = []
pvals_LRI = []
pvals_dist = []
for pair in list(all_distances.columns):
    if all_LR_intersects[pair].sum()==0:
        pvals_LR.append(1)
    else:
        pvals_LR.append((all_LR_intersects[pair]>real_LR_intersects[pair][0]).sum()/10)
    pvals_LRI.append((all_LRI_intersects[pair]>real_LRI_intersects[pair][0]).sum()/10)
    pvals_dist.append((all_distances[pair]<real_distances[pair][0]).sum()/10)

pd.DataFrame(np.asarray(pvals_LR).reshape(18,18), columns = disease_list, index = disease_list)#.to_csv('processed_data/02a_bootstrapped_disease_comparisons/pvals_overlapping_LRs.csv')
pd.DataFrame(np.asarray(pvals_LRI).reshape(18,18), columns = disease_list, index = disease_list)#.to_csv('processed_data/02a_bootstrapped_disease_comparisons/pvals_overlapping_LRIs.csv')
pd.DataFrame(np.asarray(pvals_dist).reshape(18,18), columns = disease_list, index = disease_list)#.to_csv('processed_data/02a_bootstrapped_disease_comparisons/pvals_deltacon_distances.csv')